In [1]:
# import wget
# for i in range(1, 8):
#     for letter in ['c', 'w', 'p', 's']:
#         url = f'https://people.sc.fsu.edu/~jburkardt/datasets/knapsack_01/p0{i}_{letter}.txt'
#         wget.download(url, 'D:\Study\AIO\Lab2\\benchmarks')

In [2]:
def count_weight(indexes: list, weights: list):
    """Функция, которая считает общий вес предметов в рюкзаке"""
    weight = 0
    assert len(indexes) == len(weights)
    for i in range(len(weights)):
        weight += weights[i] * indexes[i]
    return weight

In [3]:
import numpy as np

def knapsack(method, cap, p, w):
    """Функция, которая возвращает индексы предметов, время и количество операций
        метода method решения задачи о рюкзаке"""
    times = []
    for i in range(10):
        import time
        start = time.time()
        indexes, profit, ops_cnt = method(cap, p, w)
        end = time.time()
        time = end - start
        times.append(time)
    return indexes, profit, ops_cnt, np.mean(times)

In [4]:
def dp(cap: int, p: list, w: list):
    """Метод решения задачи о рюкзаке с помощью ДП на весах"""
    n = len(w)
    dp = [[0 for i in range(cap + 1)] for i in range(n + 1)]
    items = []
    ops_cnt = 0
    for i in range(1, n + 1):
        for j in range(1, cap + 1):
            wi = w[i - 1]
            if wi <= j:
                dp[i][j] = max(dp[i - 1][j], p[i - 1] + dp[i - 1][j - wi])
            else:
                dp[i][j] = dp[i - 1][j]
            ops_cnt += 1
                
    def findAns(dp: list, i: int, j: int, w: list):
        if dp[i][j] == 0: 
            return
        if dp[i - 1][j] == dp[i][j]:
            findAns(dp, i - 1, j, w)
        else:
            findAns(dp, i - 1, j - w[i - 1], w)
            items.append(i - 1)
            
    findAns(dp, n, cap, w)        
    indexes = []
    for i in range(n):
        if i in items:
            indexes.append(1)
        else:
            indexes.append(0)
    
    return indexes, dp[n][cap], ops_cnt

In [112]:
from itertools import chain, combinations

def ptas(cap, p, w, k : int = 7):
    '''Knapsack task via PTAS algo by Sanhi'''
    n_items = len(w)
    ops = 0

    def powerset_k(seq, k = k):
        return chain.from_iterable(combinations(seq, r) for r in range(0, min(k, n_items)+1))

    def GS(subset):
        '''Greedy Search procedure'''
        ops_ = 0
        z_g = 0
        new_cap = cap - sum([w[j] for j in subset])
        X = []
        for j in range(0,n_items):
            if not (j in subset) and w[j] <= new_cap:
                z_g = z_g + p[j]
                new_cap = new_cap - w[j]
                X.append(j)
            ops_ = ops_+1
        return z_g, X, ops_
    
    z_h = 0
    X_h = []

    for subset in powerset_k(seq = range(n_items)):
        if sum([w[j] for j in subset]) <= cap:

            z_g, X, gs_ops= GS(subset)
            ops = ops + gs_ops

            if z_g + sum([p[j] for j in subset]) > z_h:
                z_h = z_g + sum([p[j] for j in subset])
                X_h = list(set(X + list(subset)))
    
    idx = np.zeros(n_items, dtype=int)
    idx[X_h] = 1
    return list(idx), z_h, ops

In [113]:
files_list = []
for i in range(1, 8):
    files_list.append((f'./benchmarks/p0{i}_c.txt', f'./benchmarks/p0{i}_w.txt', f'./benchmarks/p0{i}_p.txt', f'./benchmarks/p0{i}_s.txt'))

In [114]:
for i, sample in enumerate(files_list):
    with open(sample[0], 'r') as c, open(sample[1], 'r') as w, open(sample[2], 'r') as p, open(sample[3], 'r') as s:
        capacity = int(c.read())
        weights = list(map(int, w.read().split()))
        profits = list(map(int, p.read().split()))
        answer = list(map(int, s.read().split()))
        print(f'Sample {i + 1}')
        
        print('--------------------')
        print('DP on weights:')
        indexes, profit, operations, time = knapsack(method=dp, cap=capacity, p=profits, w=weights)
        weight = count_weight(indexes, weights)
        print(f'Capacity: {capacity}')
        print(f'Items: {indexes}')
        print(f'Profit: {profit}')
        print(f'Weight: {weight}')
        print(f'Time: {time}')
        print(f'Operations count: {operations}')
        assert answer == indexes
        print('--------------------')
        print(f'Right answer: {answer}')
        
        print('\n\n')

Sample 1
--------------------
DP on weights:
Capacity: 165
Items: [1, 1, 1, 1, 0, 1, 0, 0, 0, 0]
Profit: 309
Weight: 165
Time: 0.001375579833984375
Operations count: 1650
--------------------
Right answer: [1, 1, 1, 1, 0, 1, 0, 0, 0, 0]



Sample 2
--------------------
DP on weights:
Capacity: 26
Items: [0, 1, 1, 1, 0]
Profit: 51
Weight: 26
Time: 0.00014269351959228516
Operations count: 130
--------------------
Right answer: [0, 1, 1, 1, 0]



Sample 3
--------------------
DP on weights:
Capacity: 190
Items: [1, 1, 0, 0, 1, 0]
Profit: 150
Weight: 190
Time: 0.0010169506072998046
Operations count: 1140
--------------------
Right answer: [1, 1, 0, 0, 1, 0]



Sample 4
--------------------
DP on weights:
Capacity: 50
Items: [1, 0, 0, 1, 0, 0, 0]
Profit: 107
Weight: 50
Time: 0.00030214786529541015
Operations count: 350
--------------------
Right answer: [1, 0, 0, 1, 0, 0, 0]



Sample 5
--------------------
DP on weights:
Capacity: 104
Items: [1, 0, 1, 1, 1, 0, 1, 1]
Profit: 900
Weight: 104

In [115]:
for i, sample in enumerate(files_list):
    with open(sample[0], 'r') as c, open(sample[1], 'r') as w, open(sample[2], 'r') as p, open(sample[3], 'r') as s:
        capacity = int(c.read())
        weights = list(map(int, w.read().split()))
        profits = list(map(int, p.read().split()))
        answer = list(map(int, s.read().split()))
        print(f'Sample {i + 1}')
        
        print('--------------------')
        print('PTAS:')
        indexes, profit, operations, time = knapsack(method=ptas, cap=capacity, p=profits, w=weights)
        weight = count_weight(indexes, weights)
        print(f'Capacity: {capacity}')
        print(f'Items: {indexes}')
        print(f'Profit: {profit}')
        print(f'Weight: {weight}')
        print(f'Time: {time}')
        print(f'Operations count: {operations}')
        assert answer == indexes
        print('--------------------')
        print(f'Right answer: {answer}')
        
        print('\n\n')

Sample 1
--------------------
PTAS:
Capacity: 165
Items: [1, 1, 1, 1, 0, 1, 0, 0, 0, 0]
Profit: 309
Weight: 165
Time: 0.0023995637893676758
Operations count: 1420
--------------------
Right answer: [1, 1, 1, 1, 0, 1, 0, 0, 0, 0]



Sample 2
--------------------
PTAS:
Capacity: 26
Items: [0, 1, 1, 1, 0]
Profit: 51
Weight: 26
Time: 0.00012946128845214844
Operations count: 90
--------------------
Right answer: [0, 1, 1, 1, 0]



Sample 3
--------------------
PTAS:
Capacity: 190
Items: [1, 1, 0, 0, 1, 0]
Profit: 150
Weight: 190
Time: 0.00024123191833496093
Operations count: 204
--------------------
Right answer: [1, 1, 0, 0, 1, 0]



Sample 4
--------------------
PTAS:
Capacity: 50
Items: [1, 0, 0, 1, 0, 0, 0]
Profit: 107
Weight: 50
Time: 0.0009106636047363281
Operations count: 497
--------------------
Right answer: [1, 0, 0, 1, 0, 0, 0]



Sample 5
--------------------
PTAS:
Capacity: 104
Items: [1, 0, 1, 1, 1, 0, 1, 1]
Profit: 900
Weight: 104
Time: 0.0014869213104248048
Operations count: